# Running MegaDetector on camera trap images

[Open this notebook in Colab](https://colab.research.google.com/github/agentmorris/MegaDetector/blob/main/notebooks/megadetector_colab.ipynb)

Adapted from previous versions by [@louis030195](https://github.com/louis030195)
and [@alsnothome](https://github.com/alsnothome).

Also see the [MegaDetector guide on GitHub](https://github.com/agentmorris/MegaDetector/blob/main/megadetector.md) and the [MegaDetector Python package documentation](https://megadetector.readthedocs.io).

This notebook is designed to load camera trap images that have already been uploaded to Google Drive. If you don't have your own images on Google Drive, this notebook will show you how to download some sample images from [LILA](https://lila.science).

MegaDetector output is saved in a .json file whose format is described  [here](https://github.com/agentmorris/MegaDetector/tree/main/megadetector/api/batch_processing#batch-processing-api-output-format). The last cell in this notebook will give you some pointers on how users typically work with MegaDetector output.

## Set up the Colab instance to run on a GPU accelerator


Navigate to Edit→Notebook Settings and select "GPU" from the "Hardware accelerator" drop-down menu.

## Install the MegaDetector Python package

This may take 2-3 minutes.  You may be asked to re-start the Colab runtime, that's OK.

In [ ]:
pip install megadetector

## Mount Google Drive in Colab

You can skip this cell if you are running this notebook locally, and you don't need to access Google Drive.

You can mount your Google Drive if you have your sample images there, or if want to save the results to your Google Drive.  

Once you run the cell below, you will be prompted to authorize Colab to access your Google Drive.  Your Google Drive folders will then be mounted under `/content/drive` and can be viewed and navigated in the Files pane in Colab.

The method is described in [this Colab code snippet](https://colab.research.google.com/notebooks/io.ipynb#scrollTo=u22w3BFiOveA).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Download sample images

We install Microsoft's [azcopy](https://docs.microsoft.com/en-us/azure/storage/common/storage-use-azcopy-v10) utility, which we then use to download a few camera trap images from the [Snapshot Serengeti](http://lila.science/datasets/snapshot-serengeti) dataset hosted on [lila.science](http://lila.science).  If you are using your own data, you can skip this step, and instead use the cell that follows to set "LOCAL_DIR" to the folder that contains your images.

In [ ]:
%%bash

# Download azcopy
wget -q -O azcopy_linux.tar.gz https://aka.ms/downloadazcopy-v10-linux
tar -xvzf azcopy_linux.tar.gz --wildcards */azcopy --strip 1b
rm azcopy_linux.tar.gz
chmod u+x azcopy

# Copy a few Snapshot Serengeti images to a local directory
DATASET_URL = "https://lilawildlife.blob.core.windows.net/lila-wildlife/snapshotserengeti-unzipped/"
SELECTED_FOLDER = "S1/D05/D05_R4"
LOCAL_INPUT_DIR = "/content/snapshotserengeti"

./azcopy cp "${DATASET_URL}${SELECTED_FOLDER}" "${LOCAL_INPUT_DIR}" --recursive

## ...or use your own images.

If you didn't run the previous cell, use this cell to point this notebook to your images.  You'll need to change a few paths below as well.

In [ ]:
# If someone runs this whole notebook, make sure we don't over-write the "LOCAL_DIR" variable 
# set in the previous cell.
try:
    _ = LOCAL_INPUT_DIR
except:
    LOCAL_INPUT_DIR = '/path/to/your/images' # Or, in Windows: r'c:\path\to\your\images'

## Run the detection script

This step executes the Python script `run_detector_batch.py` from the MegaDetector package. It has three mandatory arguments and one optional:

1. A MegaDetector model file (this can be a model name, like "MDV5A", or a path to a model file).
2. A folder containing images.  This notebook points to the folder where we just put our Snapshot Serengeti images; if your images were already on Google Drive, replace `[Image_Folder]` with your folder name.
3. The output JSON file location and name.

There are actually two variants of MegaDetector v5, called "v5a" and "v5b".  By default this notebook runs MDv5a; change "MDV5A" to "MDV5B" below to run MDv5b instead.

Both run at the same speed; if you are in a Colab session with a GPU accelerator, you should be able to process around four images per second.

Here we are running MegaDetector using python -m to invoke the module as if we were running it at the command line.  You can call this directly via Python code as well; documentation for this module is available [here](https://megadetector.readthedocs.io/en/latest/detection.html#module-megadetector.detection.run_detector_batch).


In [ ]:
# Make sure the local input folder got set correctly
import os
assert os.path.isdir(LOCAL_INPUT_DIR)

# Choose a folder of images to process
images_dir = LOCAL_INPUT_DIR

# Choose a location for the output JSON file
output_file_path = '/content/drive/My Drive/snapshotserengeti-test/snapshot-serengeti-megadetector-results.json'

# Run MegaDetector
!python -m megadetector.detection.run_detector_batch "MDV5A" "$images_dir" "$output_file_path" --recursive --output_relative_filenames --quiet

## Visualize batch processing script outputs

Here we use the `visualize_detector_output.py` in the `visualization` folder of the MegaDetector repo to see the output of the MegaDetector visualized on our images. It will save images annotated with the results (original images will *not* be modified) to the folder you specify here.

In [ ]:
# Render bounding boxes on our images
visualization_dir = '/content/visualized_images'
!python -m megadetector.visualization.visualize_detector_output "$output_file_path" "$visualization_dir" --confidence 0.2 --images_dir "$images_dir"

In [ ]:
# Show the images with bounding boxes in Colab
import os
from PIL import Image

for viz_file_name in os.listdir(visualization_dir):
  print(viz_file_name)
  im = Image.open(os.path.join(visualization_dir, viz_file_name))
  display(im)

# Next steps

Now that you have run MegaDetector on a few images, here are some pointers to help you take advantage of MegaDetector to label your survey images more quickly.

### Ways to use the output .json in a camera trap image processing workflow

#### 1. Timelapse

[Timelapse](http://saul.cpsc.ucalgary.ca/timelapse/pmwiki.php?n=Main.HomePage) is an open-source tool for annotating camera trap images. We have worked with the Timelapse developer to integrate MegaDetector results into Timelapse, so a user can:

- Select or sort images based on whether they contain animal or people or vehicles.
- View bounding boxes during additional manual annotation steps

See the [Timelapse Image Recognition Guide](https://saul.cpsc.ucalgary.ca/timelapse/uploads/Guides/TimelapseImageRecognitionGuide.pdf) for more information.

![Screenshot showing the Timelapse application with MegaDetector output, shown as a bounding box around the detected animal](https://github.com/agentmorris/MegaDetector/blob/main/megadetector/api/batch_processing/integration/images/tl_boxes.jpg?raw=1)


#### 2. Separating images into folders that contain animals/people/vehicles/nothing

Some MegaDetector users do image review without Timelapse, by moving the images to separate folders containing animals/people/vehicles/nothing according to MegaDetector output. You can use the script [separate_detections_into_folders.py](https://github.com/agentmorris/MegaDetector/blob/main/megadetector/postprocessing/separate_detections_into_folders.py) for this.

